In [153]:
from keystonexl.client import PredatorClient
from keystonexl.base import PitchSide
import numpy as np
import enum

In [3]:
base_path = '/Users/jason.katz/Downloads'
predator_client = PredatorClient.from_paths(moment_path='{}/SAMPLE.hudlproj/momentsSave.json'.format(base_path), 
                                            tracking_path='{}/SAMPLE.hudlproj/trackingSave.json'.format(base_path))

In [203]:
test = Game(predator_client)

In [217]:
predator_client.teams

[{'hudl_team_id': 't_56465b0e-0479-44e0-bd27-02b2ae26bdb4',
  'team_name': 'West Ham United',
  'team_source_id': '21',
  'team_type': 'home'},
 {'hudl_team_id': 't_261b14ac-f2cf-41f8-9130-1bee8ce9a5e2',
  'team_name': 'Southampton',
  'team_source_id': '20',
  'team_type': 'away'}]

In [205]:
test.shots[0].team

11.81

In [172]:
class AlwaysTrue(enum.Enum):
    yes = 1
    no = 0

class FreeKick(enum.Enum):
    yes = 1
    no = 0
    
class Corner(enum.Enum):
    yes = 1
    no = 0
    
class Cross(enum.Enum):
    yes = 1
    no = 0
    
class ThrowIn(enum.Enum):
    yes = 1
    no = 0
    
class Header(enum.Enum):
    yes = 1
    no = 0
    
class Made(enum.Enum):
    yes = 1
    no = 0

In [197]:
class Game(object):
    def __init__(self, predator_client):
        self.predator_client = predator_client
        self.game_id = np.random.rand()
        self.shots = [moment for moment in self.predator_client.moments if self.predator_client.has_tag(moment, 'shot')]
        self.first_shot = self.shots[0]   
        self.first_shot_period_num = self.first_shot['period_num']
        self.first_shot_team_id = predator_client.get_value(self.first_shot, 'team_id')
        self.first_shot_team_type = predator_client.team_type(self.first_shot_team_id)
        self.first_shot_goal_side = predator_client.estimate_defending_half(self.first_shot_team_type, 
                                                                            self.first_shot_period_num)
        self.shots = [Shot(shot, self.game_id, self.predator_client, self.first_shot_period_num, self.first_shot_team_type, 
                           self.first_shot_goal_side) for shot in self.shots]

In [202]:
class Shot(object):
    def __init__(self, shot, game_id, predator_client, first_shot_period_num, first_shot_team_type, first_shot_goal_side):
        self.shot = shot
        self.game_id = game_id
        self.predator_client = predator_client
        self.first_shot_period_num = first_shot_period_num
        self.first_shot_team_type = first_shot_team_type
        self.first_shot_goal_side = first_shot_goal_side
    
            
    @property
    def always_true(self):
        return AlwaysTrue.yes
    
    @property
    def teams(self):
        return predator_client.teams
    
    def get_team_idx(self, team_type):
        for idx, team in enumerate(self.teams):
            if team['team_type'] == team_type:
                return idx
    
    @property
    def home_team_idx(self):
        return self.get_team_idx('home')
    
    @property
    def away_team_idx(self):
        return self.get_team_idx('away')
    
    @property
    def frame(self):
        return predator_client.frame_from_time(self.shot['period_num'], self.shot['end_ms'])
    
    @property
    def period_num(self):
        return self.shot['period_num']
    
    @property
    def team_id(self):
        return predator_client.get_value(self.shot, 'team_id')
    
    @property
    def home_team_id(self):
        return self.teams[self.home_team_idx]['hudl_team_id']
    
    @property
    def home_team_name(self):
        return self.teams[self.home_team_idx]['team_name']
    
    @property
    def away_team_id(self):
        return self.teams[self.away_team_idx]['hudl_team_id']
    
    @property
    def away_team_name(self):
        return self.teams[self.away_team_idx]['team_name']
        
    @property
    def team_type(self):
        return predator_client.team_type(self.team_id)
    
    @property
    def goal_side(self):
        if self.period_num == self.first_shot_period_num: 
            if self.team_type == self.first_shot_team_type:
                return self.first_shot_goal_side
            else:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
        elif self.period_num == 1 or self.period_num == 2:
            if self.team_type == self.first_shot_team_type:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
            else:
                return self.first_shot_goal_side
        else:
            return predator_client.estimate_defending_half(self.team_type, self.period_num )
        
    @property
    def goal_x(self):
        if self.goal_side == PitchSide.right:
            return predator_client.pitch_width * -.5
        else:
            return predator_client.pitch_width * .5
        
    @property
    def goal_coords(self):
        return np.array([self.goal_x, 0])
    
    @property
    def player_id(self):
        return predator_client.get_value(self.shot, 'player_id')
        
    @property
    def player_idx(self):
        return predator_client.lookup_from_df('queryable_id', self.player_id, 'tracking_index')
    
    @property
    def player_coords(self):
        return predator_client.tracking_array[self.frame, self.player_idx, :2]
    
    @property
    def x(self):
        return self.player_coords[0]
    
    @property
    def y(self):
        return self.player_coords[1]
        
    @property
    def distance(self):
        return np.linalg.norm(self.player_coords - self.goal_coords)
    
    def has_attribute(self, event_type):
        in_list = False
        while not in_list:
            for descriptor in self.shot['tags']:
                if descriptor['key'] == event_type:
                    in_list = True
            break
        return in_list
    
    @property
    def free_kick(self):
        if self.has_attribute('free kick'):
            return FreeKick.yes
        else:
            return FreeKick.no
        
    @property
    def corner(self):
        if self.has_attribute('corner'):
            return Corner.yes
        else:
            return Corner.no
        
    @property
    def cross(self):
        if self.has_attribute('cross'):
            return Cross.yes
        else:
            return Cross.no
        
    @property
    def throw_in(self):
        if self.has_attribute('throw in'):
            return ThrowIn.yes
        else:
            return ThrowIn.no
        
    @property
    def header(self):
        if self.has_attribute('header'):
            return Header.yes
        else:
            return Header.no
        
    @property
    def made(self):
        if self.has_attribute('goal'):
            return Made.yes
        else:
            return Made.no